In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc")

SLG [I]: Checking config correctness... Config correct!


100%|██████████| 3/3 [00:00<00:00, 14.33it/s]


SLG [I]: Dataset loaded from disk (TXT files)
SLG [I]: Vocabulary loaded from disk
SLG [I]: nGrams loaded from disk (['6.json', '7.json', '2.json', '3.json', '8.json', '4.json', '5.json'])


All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


SLG [I]: Paradigmatizer loaded from disk


In [ ]:
from datasets import Dataset
from tqdm.notebook import tqdm

corpus = semiotic.corpus.train

corpus_norm = []
for sent in tqdm(corpus["text"]):
    corpus_norm.append(semiotic.syntagmatic.tokenizer.normalizer.normalize_str(sent))

corpus_norm = Dataset.from_dict({"text":corpus_norm})

In [27]:
# ng4 = slg.vocabulary.nGram.build(corpus=corpus,n=4, str_normalizer=semiotic.syntagmatic.tokenizer.normalizer.normalize_str, parallel=False, keep_in_memory=True)

ng_n = 10
ng = slg.vocabulary.nGram.build(
    corpus=corpus_norm,
    n=ng_n,
    parallel=False,
    keep_in_memory=True
    )
slg.util.save_file(ng,semiotic.paths.vocabulary / f"ngrams/{ng_n}.json")

SLG [W]: This feature is not fully implemented/tested yet
SLG: Counting 10-Grams...


  0%|          | 0/413051903 [00:00<?, ?it/s]

In [14]:
from collections import defaultdict
import networkx as nx
from ipywidgets import interact
from math import log, log2, exp
import numpy as np
from tqdm.notebook import tqdm

In [6]:
ng1 = semiotic.vocab.ng1
ng2 = semiotic.vocab.ng2
ng3 = semiotic.vocab.ng3
ng4 = semiotic.vocab.ng4
ng5 = semiotic.vocab.ng5
ng6 = semiotic.vocab.ng6
ng7 = semiotic.vocab.ng7
ng8 = semiotic.vocab.ng8
# ng9 = semiotic.vocab.ng9
# ng10 = semiotic.vocab.ng10

In [7]:
norm_factor = 1/sum(ng1.freq.values())

ng1.prob = {k:v*norm_factor for k,v in ng1.freq.items()}
ng2.prob = {k:v*norm_factor for k,v in ng2.freq.items()}
ng3.prob = {k:v*norm_factor for k,v in ng3.freq.items()}
ng4.prob = {k:v*norm_factor for k,v in ng4.freq.items()}
ng5.prob = {k:v*norm_factor for k,v in ng5.freq.items()}
ng6.prob = {k:v*norm_factor for k,v in ng6.freq.items()}
ng7.prob = {k:v*norm_factor for k,v in ng7.freq.items()}
ng8.prob = {k:v*norm_factor for k,v in ng8.freq.items()}
# ng9.prob = {k:v*norm_factor for k,v in ng9.freq.items()}
# ng10.prob = {k:v*norm_factor for k,v in ng10.freq.items()}

# ng_prob = {**ng1.prob,**ng2.prob,**ng3.prob, **ng4.prob, **ng5.prob, **ng6.prob, **ng7.prob}
ng_prob = [ng1.prob, ng2.prob, ng3.prob, ng4.prob, ng5.prob, ng6.prob, ng7.prob, ng8.prob]#, ng9.prob, ng10.prob]

In [8]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [69]:
def build_graph_data(string,ng_prob):

    edge_data = []
    for beginning in range(0, len(string)):
        for end in range(beginning + 1, len(string) + 1):
            subsequence_label = string[beginning:end]
            if len(subsequence_label)>len(ng_prob) or subsequence_label not in ng_prob[len(subsequence_label)-1] or subsequence_label == string:
                continue
            edge_data.append(
                (
                    beginning,
                    end,
                    {
                        "label": subsequence_label,
                    },
                )
            )

    return edge_data

In [139]:
from semiolog.util import subsequences
from math import log2
from itertools import islice


def chain2seq(string, ng_prob):
    
    lSt = len(string)

    # If sent of len <2r, then return the interval
    if lSt<2:
        return [(0,lSt)]
    elif lSt == 2:
        return [(0,1),(1,2)]

    # If a character in the string not in vocab, add it
    for c in string:
        if c not in ng_prob[0]:
            ng_prob[0][c]=0
            # self.voc_rank[c]=(len(self.voc)+1)**self.zipf_factor

    graph_data = build_graph_data(string, ng_prob)
    seg_graph_full = nx.DiGraph()
    seg_graph_full.add_edges_from(graph_data)

    # Construct weights
    for edge in seg_graph_full.edges:
        label = seg_graph_full.edges[edge]["label"]
        score = ng_prob[len(label)-1][label]
        
        if score == 0:
            score = 0
        else:
            score = -log(score)


        seg_graph_full.edges[edge]["weight"] = score

    # Find best segmentation out of shortest path


    # def k_shortest_paths(G, source, target, k, weight=None):
    #     k_s = list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))
    #     k_l = [[string[l:r] for l,r in subsequences(sp, 2)] for sp in k_s]
    #     k_sc = [[log(ng_prob[len(label)-1][label]) for label in k_] for k_ in k_l]
    #     return (k_s,k_l,k_sc)

    # k_short = k_shortest_paths(seg_graph_full, 0, lSt, 5)

    # shortest_path = nx.shortest_path(seg_graph_full, 0, lSt, weight="weight", method="dijkstra")
    shortest_path = nx.shortest_path(seg_graph_full, 0, lSt, weight="weight", method="bellman-ford")
    shortest_path_length = nx.shortest_path_length(seg_graph_full, 0, lSt, weight="weight",method="bellman-ford")
    # shortest_path_length = nx.shortest_path_length(seg_graph_full, 0, lSt, weight="weight", method="dijkstra")

    seg_offsets = subsequences(shortest_path, 2)

    splits = []
    for start,end in seg_offsets:
        splits.append(string[start:end])

    return (splits, exp(-shortest_path_length))

In [158]:
from random import randint
sent = semiotic.corpus.train[randint(0,100000)]["text"]
# sent = "I have made my plans and I have to stick to them"
# sent = "thisisverygood"
print(sent)
sent = semiotic.syntagmatic.tokenizer.normalizer.normalize_str(sent)

And then there was his secrecy , his concealments , his clandestine behaviour .


In [159]:
chain2seq(sent,ng_prob)

(['andthen',
  'therewas',
  'his',
  'secrecy',
  'hisconce',
  'al',
  'ments',
  'his',
  'clandest',
  'in',
  'e',
  'behaviou',
  'r'],
 9.753619874057792e-46)

In [136]:
len_sent = len(sent)
pws = list(powerset(range(len_sent+1)))
possible_cuts = [i for i in pws if 0 in i and len_sent in i]
possible_offsets = []
for cut in tqdm(possible_cuts):
    possible_offsets.append(list(zip(cut,cut[1:])))
possible_offsets_n = []
for offset in possible_offsets:
    if max([r-l for l,r in offset])<9 and offset != [(0,len(sent))]:
        possible_offsets_n.append(offset)
seg_probs = []
for offsets in tqdm(possible_offsets_n):
    segs = [sent[l:r] for l,r in offsets]
    seg_probs.append((np.prod([ng_prob[len(seg)-1][seg] for seg in segs]), segs))

sorted(seg_probs, reverse=True)[:30]

  0%|          | 0/8192 [00:00<?, ?it/s]

  0%|          | 0/8080 [00:00<?, ?it/s]

[(1.2334047306502716e-09, ['thisis', 'verygood']),
 (1.1336746360318178e-10, ['t', 'hisis', 'verygood']),
 (1.0396510461982027e-10, ['thisi', 's', 'verygood']),
 (1.0210446267371826e-10, ['this', 'is', 'verygood']),
 (5.607854089912726e-11, ['this', 'i', 's', 'verygood']),
 (4.722440266801482e-11, ['thisis', 'verygoo', 'd']),
 (4.288297282264796e-11, ['th', 'isis', 'verygood']),
 (2.205213091671237e-11, ['th', 'is', 'is', 'verygood']),
 (1.8042322396313062e-11, ['t', 'his', 'is', 'verygood']),
 (1.373847298376866e-11, ['th', 'isi', 's', 'verygood']),
 (1.3477759609945006e-11, ['thi', 'sis', 'verygood']),
 (1.3376587619711515e-11, ['thi', 's', 'is', 'verygood']),
 (1.2824876407454603e-11, ['thisis', 'v', 'erygood']),
 (1.2203233198586194e-11, ['th', 'i', 'sis', 'verygood']),
 (1.2111628553176633e-11, ['th', 'i', 's', 'is', 'verygood']),
 (1.211162855317663e-11, ['th', 'is', 'i', 's', 'verygood']),
 (1.1954058115308092e-11, ['thisis', 'very', 'good']),
 (1.1485026775147747e-11, ['thisi',

In [17]:
exp(-36.59373983654784)

1.2809751852749289e-16

In [9]:
from collections import Counter
bla = Counter(ng_prob)

In [13]:
bla.most_common(10)

[('e', 0.12283516959154912),
 ('t', 0.09144317330136345),
 ('a', 0.07986945016799842),
 ('o', 0.07554999657567639),
 ('i', 0.07229377970004593),
 ('n', 0.07001552324870404),
 ('s', 0.06436566038490976),
 ('r', 0.0612647297217823),
 ('h', 0.05187486568481037),
 ('l', 0.04127051180816118)]